# **Dream 7B 微调项目报告**

## **一、 扩散模型 (Diffusion Model) 架构分析与研究**

### **1. 核心思想**

扩散模型的设计哲学源于热力学，其核心是通过两个对称的过程实现数据生成：

*   **前向过程 (Forward Process):** **注入噪声**直至其变为纯粹噪声的“破坏”过程。
*   **逆向过程 (Reverse Process):** **去除噪声**直至恢复出结构化数据的“生成”过程。

通过学习如何逆转这个“破坏”过程，模型本质上就学会了目标数据的内在分布规律，从而能够生成全新的、高质量的样本。

### **2. 前向过程 (Forward Process / Diffusion Process)**

这是一个不可学习的、固定的马尔可夫过程，其关键特性如下：

*   **目标：** 将一个原始数据样本在 T 个时间步内逐步添加高斯噪声。
*   **过程：** 随着时间步 `t` 的推进，数据样本会逐渐丧失其结构化信息。当 `t` 趋近于 `T` 时，样本最终会收敛为纯粹的噪声。
*   **意义：** 此过程并非无用功。它为模型的训练提供了至关重要的监督信号。在任意时间步 `t`，我们都能精确地得到**“加噪后的数据”**与**“所添加的噪声”**这样成对的训练样本，这是逆向过程能够成功学习的基础。

### **3. 逆向过程 (Reverse Process / Denoising Process)**

这是**模型真正需要学习的部分**。其工作机制可以分解为：

*   **目标：** 从一个纯粹的高斯噪声样本开始，通过 T 个时间步，逐步将其还原成一个符合真实数据分布的清晰样本。
*   **核心机制：** 在每一个时间步 `t`，一个**深度神经网络**会接收当前的含噪数据 `x_t`，并被训练来**预测在该步骤中被添加的噪声 `ε`**。
*   **迭代去噪：** 通过从当前数据 `x_t` 中减去模型预测出的噪声 `ε`，我们就能得到一个稍微“干净”一点的数据 `x_{t-1}`。这个过程被迭代 T 次，最终得出清晰样本。

### **4. 与语言模型的结合（dLLM）

Dream 7B 作为一种创新的扩散型语言模型，向我们介绍了概念**离散的词元空间 (Discrete Token Space)**。它同样包含：

*   **前向过程 (Masking):** 对应于传统扩散模型的“加噪”，Dream 7B 的前向过程是将文本序列中的部分或全部词元（token）替换为特殊的 **`[MASK]`** 标记。噪声由离散的`[MASK]`词元替代连续的高斯值。
*   **逆向过程 (Predicting):** 这是模型的核心生成任务。模型接收一个被掩码的文本序列，并**并行地预测所有 `[MASK]` 位置上最有可能的真实词元**。这个去噪（预测）过程会迭代进行多次：每一次迭代，模型都会基于上一轮的预测结果，生成一个置信度更高、`[MASK]` 标记更少的新序列，直至整个文本被完整恢复。

这种在离散空间中直接进行“掩码与预测”的扩散方法，是其与传统自回归语言模型（如GPT）的关键区别，赋予了其并行生成和迭代优化的独特能力。

## 二、 准备工作：模型与数据集加载
在这一部分，我们将加载所有必需的工具、模型和数据，为后续的微调做好准备。

In [1]:
# ------------------- 这是新增CSDN的代码 -------------------
import os
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'
# ---------------------------------------------------------
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset

print("Hugging Face镜像设置成功。")
print("所有需要的库都已成功导入。")

Hugging Face镜像设置成功。
所有需要的库都已成功导入。


In [2]:
# 1. 模型名称
MODEL_NAME = "Dream-org/Dream-v0-Base-7B" 

# 2. 数据文件路径
LOCAL_DATA_FILE = "train-00000-of-00001.parquet"


print(f"模型名称设定为: {MODEL_NAME}")
print(f"本地数据文件路径设定为: {LOCAL_DATA_FILE}")

模型名称设定为: Dream-org/Dream-v0-Base-7B
本地数据文件路径设定为: train-00000-of-00001.parquet


In [3]:
# --- 加载分词器  ---
print("步骤 4.1: 开始加载分词器...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
print("分词器加载成功。")


# --- 加载模型本身 ---
print("\n步骤 4.2: 开始加载模型")
from transformers import AutoConfig
from transformers.dynamic_module_utils import get_class_from_dynamic_module
print("正在加载模型配置并注入Dropout...")
config = AutoConfig.from_pretrained(MODEL_NAME)
config.attention_dropout = 0.1
print(f"配置修改成功！Attention Dropout 已被设置为: {config.attention_dropout}")

TargetClass = get_class_from_dynamic_module(
    "modeling_dream.DreamModel",  
    MODEL_NAME
)
print(f"已成功从本地代码中手动提取出目标类：{TargetClass}")
model = TargetClass.from_pretrained(
    MODEL_NAME,
    dtype=torch.bfloat16,
    device_map="auto",
)

print("模型加载成功。")

步骤 4.1: 开始加载分词器...
分词器加载成功。

步骤 4.2: 开始加载模型
正在加载模型配置并注入Dropout...


The repository Dream-org/Dream-v0-Base-7B contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/Dream-org/Dream-v0-Base-7B .
 You can inspect the repository content at https://hf.co/Dream-org/Dream-v0-Base-7B.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


配置修改成功！Attention Dropout 已被设置为: 0.1
已成功从本地代码中手动提取出目标类：<class 'transformers_modules.Dream-org.Dream-v0-Base-7B.6572adb5535263e4d1a337b56942ba48b6dee2a9.modeling_dream.DreamModel'>


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

模型加载成功。


In [4]:
print(f"步骤 5.1: 开始从本地文件 '{LOCAL_DATA_FILE}' 加载数据集...")

dataset = load_dataset("parquet", data_files=LOCAL_DATA_FILE, split='train')

print("数据集加载成功！")


# --- 步骤 5.2 & 5.3: 探索数据集 ---
print("\n步骤 5.2: 查看数据集的整体信息...")
print(dataset)

print("\n步骤 5.3: 查看第一条数据，确认字段名...")
# 打印第一条数据
print(dataset[0])

步骤 5.1: 开始从本地文件 'train-00000-of-00001.parquet' 加载数据集...
数据集加载成功！

步骤 5.2: 查看数据集的整体信息...
Dataset({
    features: ['solution', 'question', 'cot_type', 'source_type', 'metadata', 'gemini_thinking_trajectory', 'gemini_attempt', 'deepseek_thinking_trajectory', 'deepseek_attempt', 'gemini_grade', 'gemini_grade_reason', 'deepseek_grade', 'deepseek_grade_reason'],
    num_rows: 1000
})

步骤 5.3: 查看第一条数据，确认字段名...
{'solution': '128', 'question': 'Given a rational number, write it as a fraction in lowest terms and calculate the product of the resulting numerator and denominator. For how many rational numbers between 0 and 1 will $20_{}^{}!$ be the resulting product?', 'cot_type': 'math', 'source_type': 'qq8933/AIME_1983_2024', 'metadata': "{'ID': '1991-5', 'Year': 1991, 'Problem Number': 5, 'Part': None}", 'gemini_thinking_trajectory': '\nThe problem asks for the number of rational numbers between 0 and 1 such that when the rational number is written as a fraction in lowest terms, the product of t

In [5]:
print("步骤 6.1: 定义数据预处理函数")

import torch 

def preprocess_function(examples):
    questions = examples['question']
    thinking_trajectories = examples['gemini_thinking_trajectory']
    solutions = examples['solution']
    
    prompts = [
        f"Question: {q}\n\nLet's think step by step.\n{t}\n\nTherefore, the final answer is: {s}{tokenizer.eos_token}"
        for q, t, s in zip(questions, thinking_trajectories, solutions)
    ]
    
    model_inputs = tokenizer(prompts, max_length=1024, truncation=True, padding="max_length")
    
    model_inputs['attention_mask'] = torch.tensor(model_inputs['attention_mask'], dtype=torch.float32)
    # -----------------------------------------------------------

    model_inputs["labels"] = model_inputs["input_ids"]
    
    return model_inputs

print("函数定义成功。")


print("\n步骤 6.2: 预处理数据")
processed_dataset = dataset.map(
    preprocess_function, 
    batched=True, 
    remove_columns=dataset.column_names 
)
print("所有数据预处理完成。")

步骤 6.1: 定义数据预处理函数
函数定义成功。

步骤 6.2: 预处理数据
所有数据预处理完成。


## 三、 基于Dream 7B模型进行监督微调 (SFT)
为了快速验证流程，我首先在一个极小的数据子集上进行训练。

In [6]:
print("步骤 3.1: 从已处理好的1000条数据中，只取出64条用于快速测试...")

small_train_dataset = processed_dataset.select(range(64))

print("小数据集创建成功！")
print(small_train_dataset)

步骤 3.1: 从已处理好的1000条数据中，只取出64条用于快速测试...
小数据集创建成功！
Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 64
})


### 3.1 训练参数与技术路线选择
我选择以下核心参数进行首次试航训练，我的选择逻辑如下：

- **`output_dir="./checkpoints"`**: 我将所有的训练结果（模型权重、日志等）都保存在一个名为`checkpoints`的文件夹中，便于管理和后续加载。
- **`per_device_train_batch_size=1`**: 由于7B大模型对显存占用极高，我选择从**最小的批量大小（1）**开始。这是保证训练能够启动、**防止显存溢出（OOM）**的最稳妥的策略。
- **`gradient_accumulation_steps=8`**: 每8个小批次（batch size=1）才更新一次模型权重。可以得出有意义的结果。
- **`num_train_epochs=1`**: 因为这只是一次流程验证，我只训练**一个完整的世代（epoch）**。
- **`logging_steps=1`**: 我将日志打印的步数设为**1**，这意味着**每处理一个小批次，我都能立刻在屏幕上看到损失（Loss）的变化**。

In [7]:
from transformers import TrainingArguments, Trainer

print("步骤 3.2: 配置训练参数 ")

training_args = TrainingArguments(
    output_dir="./checkpoints",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    learning_rate=2e-5,
    num_train_epochs=1,
    logging_steps=1,
    save_strategy="no",
    bf16=True,   
    optim="paged_adamw_8bit",
)

print("训练参数配置完毕。")

步骤 3.2: 配置训练参数 
训练参数配置完毕。


In [8]:
print("步骤 3.3: 初始化训练器")

trainer = Trainer(
    model=model,                  
    args=training_args,           
    train_dataset=small_train_dataset, 
)

print("训练器初始化完毕。")


print("\n--- 开始首次训练 ---")
trainer.train()
print("--- 首次训练完成。---")

步骤 3.3: 初始化训练器
训练器初始化完毕。

--- 开始首次训练 ---


`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
1,0.363100
2,0.039300
3,0.016300
4,0.013800
5,0.010400
6,0.216200
7,0.014200
8,0.010400


--- 首次训练完成。---


### 发现 #1：两种微调技术路线的端到端流程验证与对比分析

- **实验目的：**
  为验证整个技术管线的正确性，并对不同的微调策略进行评估，我在64个样本的小数据集上，分别使用**全参数微调 (Full Fine-Tuning, FFT)** 和 **QLoRA** 两种技术路线。

- **实验数据与分析：**
  两次试验的训练损失（Training Loss）被记录如下。通过对这组并行数据的细致分析，可以清晰地揭示两种范式的内在特性与差异。

| 训练步骤 (Step) | 全参数微调 Loss | QLoRA Loss | 数据驱动的分析解读 |
| :---: | :---: | :---: | :--- |
| 1 | 0.3631 | 0.3216 | **初始损失 (Initial Loss):** 两者初始Loss在同一数量级，表明模型起点状态一致。QLoRA因4-bit量化引入的微小精度损失，其初始值略低，这在理论上是可能的。 |
| 2 | **0.0393 (↓)** | 0.2925 | **收敛速度 (Convergence Speed):** FFT展现出极高的收敛速度，Loss在一个步骤内下降了近90%。这体现了调动全部参数进行更新的巨大效能。QLoRA的下降则更为平缓。 |
| 3 | 0.0163 | 0.2851 | **收敛轨迹 (Convergence Trajectory):** FFT在极少的步骤内迅速将Loss降低至一个很低的水平。QLoRA则呈现出一种更稳定、更渐进的下降轨迹。 |
| 4 | 0.0138 | 0.2829 | **持续优化:** 两种方法都在持续学习，FFT已进入精细调优阶段，而QLoRA仍存续在主要下降通道中。 |
| 5 | 0.0104 | 0.2611 | **阶段性低点:** FFT的Loss已接近0.01。QLoRA稳定地达到了其在此次试验中的性能较优区间。 |
| 6 | 0.2162 (↑) | 0.5439 (↑)| **损失波动性 (Loss Volatility):** 两者都在同一步骤遭遇了损失的大幅反弹，这符合小批量随机梯度下降的特性，表明该数据批次可能是一个“困难样本”(hard case)。 |
| 7 | 0.0142 | 0.2613 | **鲁棒性与恢复:** 两者都表现出良好的鲁棒性，在遭遇损失尖峰后能迅速恢复，将Loss拉回到正常水平。 |
| 8 | 0.0104 | 0.3337 (↑)| **最终训练损失 (Final Training Loss):** FFT最终收敛在一个极低的Loss区间，展现了其在小样本上的**极高模型容量和拟合能力**。QLoRA则收敛在一个相对较高的Loss区间。 |

- **结论与评估：**
  1.  **流程验证成功：** 两次试验的成功运行，证明了环境配置、模型加载、数据预处理，到两种不同范式的训练参数设置是正确的。
  2.  **范式差异的量化体现：**
      *   **全参数微调 (FFT)** 在理论上拥有无与伦比的**学习效率和拟合能力**。它能够在极少的训练样本上，迅速将训练损失降低到一个极低的水平。
      *   **QLoRA** 通过冻结绝大部分模型参数，本质上扮演了一个**强正则化器**的角色。这限制了模型在小样本上的极致拟合能力，因此其训练损失显著高于FFT。然而，这种限制恰恰是其优势所在：它不仅**极大地降低了资源消耗**，使得微调成为可能。

## 四、 全量数据微调 
本章节将使用1000条S1K高质量样本，对`Dream-7B`模型进行一次完整的监督微调。

*   **`per_device_train_batch_size=1`**:
    将每个设备（GPU）的批量大小设为1，降低单步训练中显存占用。

*   **`gradient_accumulation_steps=8`**:
    每8个物理批次（`batch size=1`）才进行一次梯度更新。

*   **`bf16=True`**:
    将模型显存占用减半，提升训练速度。

*   **`optim="paged_adamw_8bit"`**:
    降低显存消耗

*   **`num_train_epochs=3`**:
    我们将对全部1000条数据进行3个世代的完整训练。这个设置旨在确保模型有足够的时间从数据中进行充分的学习，同时不过度增加过拟合的风险。

*   **`learning_rate=2e-5`**:
    `2e-5` (0.00002) 是在微调大型Transformer模型时，一个经过大量实践验证的、非常安全且有效的初始学习率，能够在保证收敛的同时，避免训练初期的不稳定。

*   **`logging_steps=5`**:
    我将日志记录的频率设置为每5步一次，以采集到足够高分辨率的实验数据。

*   **`save_strategy="no"`**:
    该模型的自定义代码与`transformers`库的保存逻辑存在API不兼容问题，避免崩溃。

In [6]:
from transformers import TrainingArguments, Trainer

print("步骤 4.1: 配置全参数微调的参数")

fft_full_training_args = TrainingArguments(
    output_dir="./checkpoints_fft_full_attempt", # 一个新的目录记录
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    num_train_epochs=3,                  
    logging_steps=5,                     # 更频繁地记录数据（每5步一次）
    save_strategy="no",                  # 关闭保存避免已知错误
    bf16=True,                          
    optim="paged_adamw_8bit", 
    weight_decay=0.01,
)

print("参数配置完毕。")

步骤 4.1: 配置全参数微调的参数
参数配置完毕。


In [7]:
print("步骤 4.2: 初始化")

fft_final_trainer = Trainer(
    model=model,                       
    args=fft_full_training_args,       
    train_dataset=processed_dataset,   
)

print("训练器初始化完毕。")

步骤 4.2: 初始化
训练器初始化完毕。


In [8]:
fft_final_trainer.train()

`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
5,0.228400
10,0.012200
15,0.003300
20,0.001500
25,0.035400
30,0.004900
35,0.001500
40,0.000500
45,0.006500
50,0.000700


TrainOutput(global_step=375, training_loss=0.004691547965805512, metrics={'train_runtime': 1913.0188, 'train_samples_per_second': 1.568, 'train_steps_per_second': 0.196, 'total_flos': 1.30325651914752e+17, 'train_loss': 0.004691547965805512, 'epoch': 3.0})

### 发现 #2：全量数据训练过程中的学习动态与过拟合的最终验证

在对`Dream-7B`模型应双重Dropout + 权重减）后，我成功地在全部1000条样本上，进行了一次完整的、3个世代的全参数微本。

#### 学习过程的阶段性分析

通过对375个训练步骤的日志进行分析，可以将整个学习过程清晰地划分为三个 distinct 阶段：

1.  **第一阶段：快速收敛与高度波动期 (Steps 5-130)**
    *   **观察：** 训练在启动后，展现了极高的学习效率。损失值从初始的`0.1774`，在极短的时间内便迅速收敛到了`0.000xxx`的极低水平。
    *   **同时，** 这个阶段也伴随着**剧烈的损失波动**。例如，在Step 20-25和Step 50-55之间，都出现了损失的显著反弹。
    *   **解读：** 这证明了全参数微调模式巨大的模型容量使其能够飞速地拟合数据。然而，这也使其对不同数据批次的“个性”极其敏感，导致了学习过程的内在不稳定性。
.  **第二阶段：灾难性过拟合的临界点 (Steps 135-190)**
    *   **观察：** *，一个决定性的事件发生了：训练损失**第一次精确地达到了`0.000000`**。在此之后，零损失的现象开始零星出现（如Step 190）。
 合**的、无可辩驳的最终信号。它标志着模型已经从“学习规律”的阶段，跨入了**“完美记忆训练样本”**的阶段。在处理这些特定批次时，模型给出的预测与标准答案**完全一致**。

3.  **第三阶段：完全记忆与泛化能力丧失 (Steps 190-375)**
    *   **观察：** 在训练的后半程，**零损失的现象变得越来越频繁**，最终在训练的最后阶段成为常态。
    *   **解读：** 这证明了我们部署的“三位一体”正则化策略，对于一个拥有70亿自由参数、却只学习1000个样本的模型来说，其约束力**是完全不足的**。模型的巨大容量，使其过拟合的“惯性”无可阻挡。到训练结束时，可以合理地断定，该模型已经将训练集中的绝大部分样本“背诵”了下来。

#### 最终结论

这次最终的、成功的全参数微调实验，以一种定量答案驳的方式，为我的技术选型提供不适用判决书”：

*   **全参数微调的“诅咒”：** 尽管拥有上最强的学习“大模型、小数据”这一典型场景，其最化能力**为代价，换取了在训练集上的“虚假完美”。因此，尽管我们成功地让它跑完了全程，但其产出的模型，对于任何需要推理未知问题的真实世界任务来说，**是无效的**。
失败   **QLoRA的优越性：** 这次FFT的“成功”，从反面最有力地证明了QLoRA的价值。QLoRA通过其内在的、极其强大的参数冻结机制，扮演了一个**架构级别的强正则化器**。它强制模型去学习更通用、更鲁棒的规律，从而避免了这种灾难性的过拟合。因此，尽管QLoRA的训练损失更高，但可以预期，它在未知数据上的**泛化性能将远超**这个“完美拟合”的全参数微调模型。